In [7]:
import numpy as np
np.seterr(divide="ignore")

def switch_array_columns(array:np.ndarray, i1: int, i2: int) -> np.ndarray:
    new_array = array.copy()
    match new_array.shape:
        case (_,):
            new_array[[i1, i2]] = new_array[[i2, i1]]

        case (_, _):
            new_array[:,[i1, i2]] = new_array[:, [i2, i1]]

        case _:
            raise(ValueError)
            
    return new_array

def iteracion_simplex(c: np.ndarray, A: np.ndarray, b: np.ndarray, order: np.ndarray | None = None) -> dict:
    if len(c.shape) == 1:
        raise ValueError("C debe ser un vector columna")
    if len(b.shape) == 1:
        raise ValueError("b debe ser un vector columna")
    
    m, n = A.shape
    
    # Separar matriz A en variables básicas y no básicas
    B = A.copy()[:,0:m]
    N = A.copy()[:, m:n]
    
    #Costos c básicos y no básicos
    c_B = c.copy()[0:m, 0]
    c_N = c.copy()[m:n, 0]

    try:
        B_inv = np.linalg.inv(B)
    except Exception as _:
        print('singular matrix')
        i1 = np.random.randint(0, n)
        i2 = np.random.randint(0, n)
        A_new = switch_array_columns(A, i1, i2)
        return iteracion_simplex(c, A_new, b)
    
    # solución inicial z0 = z(x0)
    x0 = B_inv@b
    z0 = (c_B.T)@(B_inv@b)

    if any(x < 0 for x in x0):
        print('non basic solution')
        i1 = np.random.randint(0, n)
        i2 = np.random.randint(0, n)
        A_new = switch_array_columns(A, i1, i2)
        return iteracion_simplex(c, A_new, b)

    pi = c_B.T@B_inv
    costos_reducidos = pi@N - c_N.T
    b_barra = B_inv@b
    Y = B_inv@N

    return {
        "z": z0,
        "x": x0,
        "pi": pi,
        "c_reducidos": costos_reducidos,
        "b_barra": b_barra,
        "Y": Y,
        "order": order if order is not None else np.arange(n)
    }



In [8]:
def metodo_simplex_revisado(c: np.ndarray, A: np.ndarray, b: np.ndarray):
    # TODO implementar el orden para distinguir las variables y ver que putas cambia el resultado con distintos args
    
    m, n = A.shape
    inicial = iteracion_simplex(c, A, b)
    c_reducidos = inicial['c_reducidos']

    if all(x <= 0 for x in c_reducidos):
        return inicial
    indice_no_basicas = np.argmax(c_reducidos)
    
    Y = inicial['Y']
    b_barra = inicial['b_barra']
    cocientes = np.divide(b_barra, np.vstack(Y[:, indice_no_basicas]))

    indice_basicas = np.where(cocientes > 0, cocientes, np.inf).argmin()

    i1, i2 = indice_basicas, indice_no_basicas + m

    A_new = switch_array_columns(A, i1, i2)
    c_new = switch_array_columns(c.T, i1, i2).T
    # order_new = switch_array_columns(inicial['order'])

    return metodo_simplex_revisado(c_new, A_new, b)

### Demo

In [9]:
A = np.array([[1,0,0,1,0],[0,1,0,0,1],[0,0,1,3,2]])
c = np.vstack([0,0,0,-3,-5])
b = np.vstack([4, 6, 18])

metodo_simplex_revisado(c,A,b)

{'z': array([-36.]),
 'x': array([[2.],
        [6.],
        [2.]]),
 'pi': array([ 0., -3., -1.]),
 'c_reducidos': array([-1., -3.]),
 'b_barra': array([[2.],
        [6.],
        [2.]]),
 'Y': array([[-0.33333333,  0.66666667],
        [ 0.        ,  1.        ],
        [ 0.33333333, -0.66666667]]),
 'order': array([0, 1, 2, 3, 4])}

Con otro orden se comporta raro

In [30]:
A = np.array([[3, 2, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 0, 0, 1]])
c = np.vstack([-3,-5,0,0,0])
b = np.vstack([18, 4, 6])
metodo_simplex_revisado(c,A,b)


non basic solution


{'z': array([-42.]),
 'x': array([[4.],
        [6.],
        [6.]]),
 'pi': array([ 0., -3., -5.]),
 'c_reducidos': array([-3., -5.]),
 'b_barra': array([[4.],
        [6.],
        [6.]]),
 'Y': array([[ 1.,  0.],
        [ 0.,  1.],
        [ 3., -2.]]),
 'order': array([0, 1, 2, 3, 4])}